In [1]:
!pip install pymupdf langchain

In [2]:
import os
import fitz  # PyMuPDF for PDF processing
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [4]:
pip install pymupdf


Note: you may need to restart the kernel to use updated packages.


In [5]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    """
    Extract text from a PDF file using PyMuPDF.
    
    Parameters:
    pdf_path (str): Path to the PDF file.
    
    Returns:
    str: Extracted text from the PDF.
    """
    pdf_document = fitz.open(pdf_path)  # Open the PDF file
    text = ""
    
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)  # Load each page
        text += page.get_text()  # Extract text from the page
    
    return text


In [6]:
import os
import fitz  # PyMuPDF

# Define the extract_text_from_pdf function
def extract_text_from_pdf(pdf_path):
    """
    Extract text from a PDF file using PyMuPDF.
    
    Parameters:
    pdf_path (str): Path to the PDF file.
    
    Returns:
    str: Extracted text from the PDF.
    """
    pdf_document = fitz.open(pdf_path)  # Open the PDF file
    text = ""
    
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)  # Load each page
        text += page.get_text()  # Extract text from the page
    
    return text

# Assuming pdf_folder is the folder where your PDFs are stored
pdf_folder = "data"
pdf_texts = {}  # Dictionary to hold the extracted text

# Extract text from all PDFs in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, filename)
        
        if os.path.exists(pdf_path):  # Extra check for file existence
            pdf_texts[filename] = extract_text_from_pdf(pdf_path)
            print(f"✅ Extracted text from {filename}")
        else:
            print(f"❌ File not found: {filename}")

# Now you can proceed with generating embeddings, etc.


✅ Extracted text from Artificial Intelligence Applications in Information and Communication Technologies.pdf
✅ Extracted text from generative-ai-and-machine-learning-for-the-enterprise.pdf


In [7]:
# Initialize the text splitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# Assuming pdf_texts is a dictionary where each value is the text of a PDF
pdf_chunks = {}

# Split the text of each PDF into chunks
for filename, text in pdf_texts.items():
    chunks = splitter.split_text(text)
    pdf_chunks[filename] = chunks
    print(f"✅ Split text for {filename}")

# Now pdf_chunks will contain the text chunks


✅ Split text for Artificial Intelligence Applications in Information and Communication Technologies.pdf
✅ Split text for generative-ai-and-machine-learning-for-the-enterprise.pdf


In [8]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

pdf_chunks = {}
for filename, text in pdf_texts.items():
    pdf_chunks[filename] = splitter.split_text(text)
    print(f"✅ Split {filename} into {len(pdf_chunks[filename])} chunks")


✅ Split Artificial Intelligence Applications in Information and Communication Technologies.pdf into 1220 chunks
✅ Split generative-ai-and-machine-learning-for-the-enterprise.pdf into 49 chunks


In [9]:
import os
pdf_folder = "data"  # Ensure this folder exists and contains PDFs
pdf_texts = {}  # Dictionary to store extracted text

if not os.path.exists(pdf_folder):
    print("❌ Folder not found! Please check the directory path.")
else:
    # Loop through all PDFs in the folder
    for filename in os.listdir(pdf_folder):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, filename)
            
            if os.path.exists(pdf_path):  # Extra check for file existence
                pdf_texts[filename] = extract_text_from_pdf(pdf_path)
                print(f"✅ Extracted text from {filename}")
            else:
                print(f"❌ File not found: {pdf_path}")

# Display total PDFs processed
print(f"\n📄 Total PDFs processed: {len(pdf_texts)}")


# Split the text into chunks (this step depends on how you want to chunk the documents)
# For simplicity, let's say each page is a chunk:
chunks = pdf_texts  # Adjust this depending on how you want to chunk

# Get embeddings
embeddings = get_embeddings(chunks)

# Create FAISS index
index = create_faiss_index(embeddings)

# Save the FAISS index
save_index(index, "research_index.faiss")


✅ Extracted text from Artificial Intelligence Applications in Information and Communication Technologies.pdf
✅ Extracted text from generative-ai-and-machine-learning-for-the-enterprise.pdf

📄 Total PDFs processed: 2


NameError: name 'get_embeddings' is not defined

In [ ]:
# Display the first chunk of the first processed PDF
first_pdf = list(pdf_chunks.keys())[0]
print(f"📄 First chunk from {first_pdf}:\n")
print(pdf_chunks[first_pdf][0])

In [14]:
import fitz  # PyMuPDF for PDF reading
import numpy as np
import faiss
import ollama
from sentence_transformers import SentenceTransformer

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file"""
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

def chunk_text(text, chunk_size=512):
    """Split text into smaller chunks"""
    words = text.split()
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

class PDFEmbeddingStore:
    def __init__(self):
        """Initialize FAISS index"""
        self.text_chunks = []
        self.model = SentenceTransformer("all-MiniLM-L6-v2")  # Using a local transformer model
        self.dimension = 384  # SentenceTransformer output size
        self.index = faiss.IndexFlatL2(self.dimension)

    def generate_embedding(self, text):
        """Generate embedding using SentenceTransformer"""
        return np.array(self.model.encode(text), dtype=np.float32)

    def store_pdf(self, pdf_path):
        """Extracts text, chunks it, generates embeddings, and stores in FAISS"""
        text = extract_text_from_pdf(pdf_path)
        chunks = chunk_text(text)
        
        for chunk in chunks:
            embedding = self.generate_embedding(chunk).reshape(1, -1)
            self.index.add(embedding)
            self.text_chunks.append(chunk)
        print(f"✅ Stored {len(chunks)} chunks from {pdf_path}")

    def retrieve_similar(self, query, k=5):
        """Retrieve top-k similar texts from FAISS"""
        query_embedding = self.generate_embedding(query).reshape(1, -1)
        distances, indices = self.index.search(query_embedding, k)
        return [self.text_chunks[i] for i in indices[0] if i < len(self.text_chunks)]

if __name__ == "__main__":
    store = PDFEmbeddingStore()
    pdf_path = "generative-ai-and-machine-learning-for-the-enterprise.pdf"  # Replace with your PDF file
    
    store.store_pdf(pdf_path)
    query = "Explain AI in simple words."
    results = store.retrieve_similar(query)
    
    print("\n🔍 Most Relevant Results:")
    for res in results:
        print("-", res[:200] + "...")


✅ Stored 6 chunks from generative-ai-and-machine-learning-for-the-enterprise.pdf

🔍 Most Relevant Results:
- be highly curated to business domains and use cases that drive quick gains in productivity and time to value—such as augmenting and automating HR, customer service and code generation. 4. Empowering Y...
- Generative AI and ML for the enterprise Contents 01 → Introduction 02 → Generative AI, foundation models and ML 03 → Advantages of foundation models and ML 04 → Common use cases and tasks 05 → What to...
- form of generative AI that combines transformer architecture with unsupervised learning, a ML function. Foundation models are large AI neural networks trained on extensive unlabeled data and fine-tune...
- Consulting™ and the 21,000 data, AI and automation consultants, in addition to the Center of Excellence for Generative AI comprised of more than 1,000 consultants with specialized generative AI expert...
- build anomaly detection systems to identify unusual patterns or beh

In [16]:
import fitz  # PyMuPDF for PDF reading
import numpy as np
import faiss
import ollama
from sentence_transformers import SentenceTransformer

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file"""
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

def chunk_text(text, chunk_size=512):
    """Split text into smaller chunks"""
    words = text.split()
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

class PDFEmbeddingStore:
    def __init__(self):
        """Initialize FAISS index"""
        self.text_chunks = []
        self.model = SentenceTransformer("all-MiniLM-L6-v2")  # Using a local transformer model
        self.dimension = 384  # SentenceTransformer output size
        self.index = faiss.IndexFlatL2(self.dimension)

    def generate_embedding(self, text):
        """Generate embedding using SentenceTransformer"""
        return np.array(self.model.encode(text), dtype=np.float32)

    def store_pdf(self, pdf_path):
        """Extracts text, chunks it, generates embeddings, and stores in FAISS"""
        text = extract_text_from_pdf(pdf_path)
        chunks = chunk_text(text)
        
        for chunk in chunks:
            embedding = self.generate_embedding(chunk).reshape(1, -1)
            self.index.add(embedding)
            self.text_chunks.append(chunk)
        print(f"✅ Stored {len(chunks)} chunks from {pdf_path}")

    def retrieve_similar(self, query, k=5):
        """Retrieve top-k similar texts from FAISS"""
        query_embedding = self.generate_embedding(query).reshape(1, -1)
        distances, indices = self.index.search(query_embedding, k)
        return [self.text_chunks[i] for i in indices[0] if i < len(self.text_chunks)]

    def generate_response(self, query):
        """Retrieve relevant content and generate response using Ollama"""
        retrieved_chunks = self.retrieve_similar(query, k=3)
        context = "\n".join(retrieved_chunks)

        # Format prompt for LLM
        prompt = f"Context:\n{context}\n\nQuestion: {query}\n\nAnswer:"
        
        # Generate response using Ollama
        response = ollama.generate(model="gemma2:2b", prompt=prompt)
        return response['response']

if __name__ == "__main__":
    store = PDFEmbeddingStore()
    pdf_path = "generative-ai-and-machine-learning-for-the-enterprise.pdf"  # Replace with your PDF file
    
    store.store_pdf(pdf_path)
    query = "Explain AI in simple words."
    
    response = store.generate_response(query)
    
    print("\n🤖 AI Response:")
    print(response)


✅ Stored 6 chunks from generative-ai-and-machine-learning-for-the-enterprise.pdf

🤖 AI Response:
Imagine you're teaching a computer to learn by giving it lots of examples of something.  Like, if you want the computer to recognize cats, you show it pictures of cats and dogs, and it learns to spot them based on the features (like pointy ears and fluffy tails). 

**Artificial intelligence (AI) is like training computers to learn from data and then use that learning to do things like:**

* **Understand what we're saying (speech recognition):**  Think about Alexa or Siri!
* **Play games like chess (game playing AI):** Computers can now beat human champions at some complex games.
* **Recognize faces in photos (facial recognition):** Used for security, social media tagging, and more!

AI is about making computers smarter so they can solve problems we face every day, like traffic, weather forecasting, and finding new medicines! 


Does that make sense? 😊  

